In [1]:
from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import os
import wandb
import cv2

In [2]:
DATASET_PATH = Path() / '..' / '..' / '..'/ '..' / 'DATA' / 'asaginbaev' / 'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

In [3]:
# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

In [20]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [21]:
class DocDataset(Dataset):
    def __init__(self, datapacks, is_test=False, transform=None):
        self.datapacks = datapacks
        self.transform = transform
        self.data_indexes = []
        self.random_affine = torchvision.transforms.RandomAffine(90)
        self.color_jitter = torchvision.transforms.ColorJitter(2, 2)
        self.is_test = is_test
        
        for i in range(len(datapacks)):
            for j in range(len(datapacks[i])):
                if datapacks[i][j].is_test_split() == is_test:
                    self.data_indexes.append((i, j))

    def __len__(self):
        return len(self.data_indexes)

    def __getitem__(self, idx):
        
        i, j = self.data_indexes[idx]
        image = torch.FloatTensor(np.array(self.datapacks[i][j].image.convert('RGB'))) / 255
        mask = np.zeros(np.array(self.datapacks[i][j].image).shape[:2])
        mask = torch.LongTensor(cv2.fillConvexPoly(mask, np.array(self.datapacks[i][j].gt_data['quad']), (1, )))
        image = image.permute([2, 0, 1])
        return image, mask.squeeze(0)

In [22]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [23]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def use_checkpointing(self):
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)

In [24]:
train_set = DocDataset(data_packs, is_test=False)
val_set = DocDataset(data_packs, is_test=True)

In [25]:
train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=4, shuffle=False, drop_last=True, num_workers=4)

In [26]:
def dice_coeff(input, target, reduce_batch_first = False, epsilon = 1e-6):
    assert input.size() == target.size()
    assert input.dim() == 3 or not reduce_batch_first

    sum_dim = (-1, -2) if input.dim() == 2 or not reduce_batch_first else (-1, -2, -3)

    inter = 2 * (input * target).sum(dim=sum_dim)
    sets_sum = input.sum(dim=sum_dim) + target.sum(dim=sum_dim)
    sets_sum = torch.where(sets_sum == 0, inter, sets_sum)

    dice = (inter + epsilon) / (sets_sum + epsilon)
    return dice.mean()

In [27]:
def dice_loss(input, target):
    return 1 - dice_coeff(input, target, reduce_batch_first=True)

In [28]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [20]:
!nvidia-smi

Fri Mar 10 23:42:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:18:00.0 Off |                  N/A |
| 41%   47C    P2    80W / 280W |   4789MiB / 24220MiB |     34%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN V      Off  | 00000000:3B:00.0 Off |                  N/A |
| 28%   

In [13]:
logger = wandb.init(project='abbyy_ocr')

wandb: Currently logged in as: azatiusssss (use `wandb login --relogin` to force relogin)


In [14]:
criterion = nn.BCEWithLogitsLoss()

In [29]:
model = UNet(3, 1)

In [31]:
model.to(device);

In [18]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3, foreach=True)

In [19]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)

In [ ]:
global_step = 0
for epoch in tqdm(range(30)):
    for images, masks in tqdm(train_loader):
        images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        masks = masks.to(device=device, dtype=torch.long)
        masks_pred = model(images)
        loss = criterion(masks_pred.squeeze(1), masks.float())
        loss += dice_loss(F.sigmoid(masks_pred.squeeze(1)), masks.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        logger.log({
                    'train loss': loss.item(),
                    'step': global_step,
                    'epoch': epoch
                })
        global_step += 1
    for images, masks in tqdm(val_loader):
        images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        masks = masks.to(device=device, dtype=torch.long)
        with torch.no_grad():
            masks_pred = model(images)
        loss = criterion(masks_pred.squeeze(1), masks.float())
        loss += dice_loss(F.sigmoid(masks_pred.squeeze(1)), masks.float())
        logger.log({
                    'val loss': loss.item(),
                    'step': global_step,
                    'epoch': epoch
                })
        global_step += 1
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f'checkpoint_epoch_{epoch}.pth')

In [ ]:
wandb.finish()

In [22]:
model.load_state_dict(torch.load('checkpoint_epoch_24.pth'))

<All keys matched successfully>

In [32]:
masks = []
for image, _ in tqdm(val_set):
    image = image.to(device)
    with torch.no_grad():
        mask = model(image.unsqueeze(0))
    masks.append(mask.squeeze(0).cpu().detach().numpy())

  0%|          | 0/4250 [00:00<?, ?it/s]

In [34]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [41]:
def find_corners(mask):
    binary_mask = 255 * (sigmoid(mask) > 0.5).astype(np.uint8)
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = list(map(cv2.contourArea, contours))
    contour = contours[np.argmax(areas)]
    x, y = np.array(contour)[:, 0].T
    i_r_b = np.argmax(y + x)
    i_l_b = np.argmax(y - x)
    i_l_t = np.argmax(-y - x)
    i_r_t = np.argmax(-y + x)
    return np.array([[x[i_l_t], y[i_l_t],], 
                     [x[i_r_t], y[i_r_t],],
                     [x[i_r_b], y[i_r_b],],
                     [x[i_l_b], y[i_l_b],],], dtype=float)

In [42]:
results_dict = {}
for pred_mask, index in tqdm(list(zip(masks, val_set.data_indexes))):
    i, j = index
    pred_mask = pred_mask[0]
    corners = find_corners(pred_mask)
    corners[:, 0] /= pred_mask.shape[1]
    corners[:, 1] /= pred_mask.shape[0]
    
    results_dict[data_packs[i][j].unique_key] = corners

  0%|          | 0/4250 [00:00<?, ?it/s]

In [43]:
from course_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

In [44]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [45]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

/home/asaginbaev/miniconda3/envs/layout/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/asaginbaev/miniconda3/envs/layout/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/asaginbaev/miniconda3/envs/layout/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/asaginbaev/miniconda3/envs/layout/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home/asaginbaev/miniconda3/envs/layout/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/home

In [46]:
print("Точность кропа: {:1.4f}".format(acc))

Точность кропа: 0.8480
